# Práctica sobre algoritmos genéticos

<img src="imgs/tren.jpg">


Un área ferroviaria de carga/descarga con una única vía de entrada y otra salida se compone de tres muelles de carga/descarga: Op1, Op2 y Op3, correspondientes a contenedores, carbón y gas. Por tanto, cada tren que llega se dirige a un muelle en función de su carga. Un tren tarda en cargar o descargar un tiempo proporcional al número de vagones que arrastra. Cada día llegan secuencialmente n trenes. Si los trenes son de cargas distintas, pueden entrar en paralelo a los muelles. Cuando dos trenes con el mismo tipo de carga se encuentran seguidos, el segundo debe esperar por el primero, así como todos los demás que se encuentren por detrás.


Se nos plantea resolver, mediante un algoritmo genético, el problema de la ordenación en la entrada de los trenes para minimizar el tiempo de paso del conjunto de trenes.


<img src="imgs/docks.jpg" width="70%">

In [17]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

## Trenes

En el siguiente código, <code>random_trains_generation</code> genera los trenes que llegarán en un día. Cada ordenación diferente de este conjunto corresponderá con un **individuo** en nuestro eaquema de algoritmo genético.

In [18]:
import random
from deap import base, creator, tools

class Train:
    def __init__(self, wagons, op, licence_plate):
        self.wagons = wagons
        self.op = op
        self.licence_plate = licence_plate

    def __str__(self):
        return "Número de vagones:" + str(self.wagons) \
        + "\n" + "Muelle de operaciones:" + str(self.op) \
        + "\n" + "Matrícula:" + str(self.licence_plate)


def random_trains_generation(n):

    train_list = []
    
    for i in range(n):
        wagons = random.randint(10, 30)  # Cada tren puede arrastrar entre 10 y 30 vagones
        op = random.choice(["op1", "op2", "op3"])  # A cada tren se le asigna un tipo de carga
        train_list.append(Train(wagons, op, i))

    return train_list

In [19]:
trains = random_trains_generation(100)
def randomize_trains():
    new_trains = trains[:]
    random.shuffle(new_trains)
    return new_trains

#creamos una clase de tipo lista que implemente la funcion anterior y devuelva una lista de trenes aleatoria
class Trains_list(list):
    def __init__(self):
        super().__init__(randomize_trains())

In [20]:
creator.create("Fitness", base.Fitness, weights=(-1.0, )) 
creator.create("Individual", Trains_list, fitness=creator.Fitness)

c:\Users\Gabri\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Gabri\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [21]:
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [22]:
def evaltrains(individual):
    value = 0
    time1 = 0
    time2 = 0
    time3 = 0
    time_in1 = 0
    time_in2 = 0
    time_in3 = 0
    op1_state = 0
    op2_state = 0
    op3_state = 0
    for trains in individual:
        wagons = trains.wagons
        operation =  trains.op
        # if op1_state == 0 or op2_state == 0 or op3_state == 0:
        if operation == "op1" and time_in1 == 0:
            time1 += wagons 
            #con lo de los diccionarios de mas abajo, 
            #si es 0, significaria libre asi que eso
            #tiene mejor pinta
            #op1_state = 1
            time_in1 = wagons
        elif operation == "op2" and time_in2 == 0:
            time2 += wagons
            #op2_state = 1
            time_in2 = wagons
        elif operation == "op3" and time_in3 == 0:
            time3 += wagons
            #op3_state = 1
            time_in3 = wagons
        # else:
        elif operation == "op1" and time_in1 != 0:
            time1 += time_in1
            time_in1 = wagons
            time_in2 -= time_in1
            if time_in2 < 0:
                time_in2 = 0
            time_in3 -= time_in1
            if time_in3 < 0:
                time_in3 = 0
        elif operation == "op2" and time_in2 != 0:
            time2 += time_in2
            time_in2 = wagons
            time_in1 -= time_in2
            if time_in1 < 0:
                time_in1 = 0
            time_in3 -= time_in2
            if time_in3 < 0:
                time_in3 = 0
        elif operation == "op3" and time_in3 != 0:
            time3 += time_in3
            time_in3 = wagons
            time_in2 -= time_in3
            if time_in2 < 0:
                time_in2 = 0
            time_in1 -= time_in3
            if time_in1 < 0:
                time_in1 = 0
        # else:
        #     #esta parte en general no funciona bien
        #     #prob mejor usar diccionario con cada operacion y su tiempo de espera?
        #     lowest_time = min(time_in1, time_in2, time_in3)
        #     time_in1 -= lowest_time #cuidado que se puede quedar negativo
        #     if time_in1 < 0:
        #         time_in1 = 0
        #     time_in2 -= lowest_time
        #     if time_in2 < 0:
        #         time_in2 = 0
        #     time_in3 -= lowest_time
        #     if time_in3 < 0:
        #         time_in3 = 0

        #     if time_in1 == 0:
        #         time1 += lowest_time #revisar estos
        #         op1_state = 0
        #         time_in1 += trains.wagons
        #     if time_in2 == 0:
        #         time2 += lowest_time
        #         time_in2 += trains.wagons
        #         op2_state = 0
        #     if time_in3 == 0:
        #         time3 += lowest_time
        #         op3_state = 0
        #         time_in3 += trains.wagons
    value = max(time1, time2, time3) #esta parte se ve bien
    return value, 

La función de evalueción corregida

In [23]:
def evaltrains2(individual):
    time = 0
    operations = {"op1": 0, "op2": 0, "op3": 0}
    for trains in individual:
        wagons = trains.wagons
        if operations[trains.op] == 0: #si el muelle esta libre
            operations[trains.op] = wagons #pues esta en espare hasta que termine
        else:
            time_in = operations[trains.op] #el teimpo de espera del muelle que toque
            for k, v in operations.items():
                #no puede haber operaciones con valores menores a 0
                if v - time_in < 0: #si el tiempo espera es menor que el tiempo de espera del muelle, poner tiempo a 0, el tiempo no debería ser negativo
                    operations[k] = 0
                else:
                    operations[k] -= time_in #si no, restar el tiempo de espera del muelle
            time += time_in  #sumar el tiempo de espera del muelle
            operations[trains.op] = wagons #meter el tren en el muelle tras haber esperado
    time += max(operations.values()) #tardaremos el tiempo máximo entre todos los muelles
    return time,

In [24]:
def cxSet2(ind1, ind2):
    cx_index = np.random.randint(len(ind1))

    child1 = ind1[cx_index:]
    child2 = ind2[cx_index:]

    for x in ind2:
        if x not in child1:
            child1.append(x)

    for x in ind1:
        if x not in child2:
            child2.append(x)

    ind1[:] = child1
    ind2[:] = child2
    return ind1, ind2


In [25]:
def get_group(ind1, ind1_left, ind2_right):
    group = []
    for x in ind1:
        if x not in ind1_left and x not in ind2_right:
            group.append(x)
    return group

In [26]:
def cxSet3(ind1, ind2):
    cx_index = np.random.randint(len(ind1))

    ind1_right = ind1[cx_index:]
    ind2_right = ind2[cx_index:]
    ind1_left = ind1[:cx_index]
    ind2_left = ind2[:cx_index]

    #elementos de ind1 que no estan en su parte izquierda(primer trozo) ni en la parte a discriminar de ind2
    group1 = get_group(ind1, ind1_left, ind2_right)
    #elementos de ind2 que no estan en su parte izquierda(primer trozo) ni en la parte a discriminar de ind1
    group2 = get_group(ind2, ind2_left, ind1_right)
    #Creamos un nuevo lado derecho de ind1 e ind2 en donde no esten los trenes que ya estan en el trozo izquierdo que vamos a usar
    #y si no se cumple, rellenamos con un tren del individuo a tratar, los cuales si recogen las condiciones que queremos.
    new_ind1_right = []
    for x in ind2_right:
        if x not in ind1_left:
            new_ind1_right.append(x)
        else:
            new_ind1_right.append(group1.pop(0))

    new_ind2_right = []
    for x in ind1_right:
        if x not in ind2_left:
            new_ind2_right.append(x)
        else:
            new_ind2_right.append(group2.pop(0))

    ind1[:] = ind1_left + new_ind2_right
    ind2[:] = ind2_left + new_ind1_right
    return ind1, ind2

In [27]:
toolbox.register("evaluate", evaltrains2)
toolbox.register("mate", cxSet3)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [28]:
def main():
    pop = toolbox.population(n=50)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean, axis=0)
    stats.register("std", np.std, axis=0)
    stats.register("min", np.min, axis=0)
    stats.register("max", np.max, axis=0)

    #algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats, halloffame=hof)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=100, stats=stats, halloffame=hof, verbose=True)
    return pop, stats, hof

In [29]:
pop, stats, hof = main()

gen	nevals	avg      	std         	min    	max    
0  	50    	[1136.26]	[51.2221866]	[1027.]	[1249.]
1  	27    	[1097.78]	[39.82978283]	[1029.]	[1210.]
2  	34    	[1073.62]	[38.23552798]	[1014.]	[1174.]
3  	22    	[1050.8] 	[26.14651028]	[1014.]	[1124.]
4  	31    	[1036.92]	[19.20295811]	[1003.]	[1105.]
5  	39    	[1034.68]	[18.27177058]	[1003.]	[1102.]
6  	34    	[1027.22]	[13.2714581] 	[1003.]	[1062.]
7  	18    	[1023.18]	[15.25213428]	[996.] 	[1077.]
8  	29    	[1019.54]	[24.04097336]	[980.] 	[1112.]
9  	33    	[1016.12]	[27.57509021]	[981.] 	[1107.]
10 	25    	[1004.32]	[18.45257706]	[946.] 	[1071.]
11 	24    	[997.12] 	[21.26935824]	[946.] 	[1055.]
12 	30    	[992.12] 	[28.50448386]	[927.] 	[1063.]
13 	35    	[978.32] 	[26.46918208]	[927.] 	[1060.]
14 	28    	[966.98] 	[28.23153556]	[927.] 	[1062.]
15 	30    	[947.86] 	[18.62794675]	[927.] 	[996.] 
16 	29    	[942.38] 	[23.69716439]	[927.] 	[1030.]
17 	31    	[933.94] 	[19.66052899]	[927.] 	[1009.]
18 	29    	[932.44] 	[18.69883419

In [30]:
#calculate the time of the initial trains list
time_of_operations = {"op1": 0, "op2": 0, "op3": 0}
for i in trains:
    time_of_operations[i.op] += i.wagons
print("el tiempo mínimo será: ",max(time_of_operations["op1"], time_of_operations["op2"], time_of_operations["op3"]))
print("el tiempo del algoritmo será: ",hof[0].fitness.values[0])


el tiempo mínimo será:  811
el tiempo del algoritmo será:  858.0


Miramos que no hayan trenes repetidos

In [31]:
licence_plates = []
for t in hof[0]:
    licence_plates.append(t.licence_plate)
    
print("Número de trenes: ", len(hof[0]))
print(len(licence_plates))
print(licence_plates)

Número de trenes:  100
100
[34, 19, 59, 5, 29, 27, 40, 98, 63, 11, 10, 50, 20, 6, 35, 31, 51, 15, 9, 64, 88, 18, 3, 36, 89, 25, 1, 82, 38, 96, 37, 67, 78, 83, 32, 85, 57, 33, 54, 48, 95, 49, 72, 12, 79, 30, 73, 71, 77, 55, 0, 45, 66, 56, 65, 61, 43, 47, 90, 52, 62, 2, 87, 13, 84, 70, 80, 4, 68, 81, 23, 94, 92, 28, 91, 24, 14, 93, 97, 53, 60, 86, 41, 75, 46, 21, 8, 39, 44, 7, 74, 76, 16, 22, 17, 69, 42, 58, 99, 26]


Miramos cual es la configuración de trenes que menos tiempo tarda en pasar

In [32]:
licencias = []
for i in hof[0]:
    licencias.append(i.licence_plate)
print(sorted(licencias))
print(len(licencias))
print(len(set(licencias)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
100
100
